In [11]:
np.random.seed(42)

# Portfolio Planner

In this activity, you will use the iexfinance api to grab historical data for a 60/40 portfolio using `SPY` to represent the stock portion and `AGG` to represent the bonds.

In [2]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline

In [3]:
#from alpaca.stocks import get_historical_data
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

assets = api.list_assets()

# Data Collection

In this step, you will need to use the IEX api to fetch closing prices for the `SPY` and `AGG` tickers. Save the results as a pandas DataFrame

In [4]:
print(assets[0])

Asset({   'class': 'us_equity',
    'easy_to_borrow': False,
    'exchange': 'AMEX',
    'id': 'f9dd6f91-72d7-42a5-8b7e-243eb5d44f07',
    'marginable': True,
    'name': 'Envela Corporation',
    'shortable': False,
    'status': 'active',
    'symbol': 'ELA',
    'tradable': True})


In [5]:
# Subset your tickers, then pull returns data:
# Set the ticker
ticker = ["SPY","AGG"]
# Set timeframe to '1D'
timeframe = '1D'
# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-365)
# Get 1 year's worth of historical data for AAPL
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df
df.drop(columns=['open', 'high', 'low', 'volume'], level=1, inplace=True)
df.head

<bound method NDFrame.head of                                 AGG       SPY
                              close     close
time                                         
2019-11-22 00:00:00-05:00  112.8500  311.0000
2019-11-25 00:00:00-05:00  112.9800  313.3500
2019-11-26 00:00:00-05:00  113.1200  314.0200
2019-11-27 00:00:00-05:00  112.9950  315.4600
2019-11-29 00:00:00-05:00  112.9250  314.2700
2019-12-02 00:00:00-05:00  112.4800  311.6000
2019-12-03 00:00:00-05:00  113.0200  309.4700
2019-12-04 00:00:00-05:00  112.8000  311.4700
2019-12-05 00:00:00-05:00  112.6900  312.0500
2019-12-06 00:00:00-05:00  112.5400  314.8400
2019-12-09 00:00:00-05:00  112.5800  313.8700
2019-12-10 00:00:00-05:00  112.5400  313.5500
2019-12-11 00:00:00-05:00  112.8400  314.4100
2019-12-12 00:00:00-05:00  112.3600  317.1400
2019-12-13 00:00:00-05:00  112.8000  317.2800
2019-12-16 00:00:00-05:00  112.5800  319.5100
2019-12-17 00:00:00-05:00  112.5950  319.5495
2019-12-18 00:00:00-05:00  112.3800  319.5800
2019

# Monte Carlo Simulation

In this step, you will run Monte Carlo Simulations for your portfolio to model portfolio performance at different retirement ages. 

Complete the following steps:
1. Calculate the daily returns for the SPY and AGG closing prices.
2. Calculate volatility for both the SPY and AGG closing prices.
3. Find the last day's closing price for both stocks and save those as variables.
4. Run a Monte Carlo Simulation of at least 500 iterations and generate at least 30 years of closing prices

### HINTS:
There are 252 trading days per year, so the number of records to generate for each Monte Carlo run will be 252 days * 30 years

In [6]:
# Calculate the daily roi for the stocks
# YOUR CODE HERE

daily_returns_SPY = df.pct_change()
daily_returns_SPY.head()
daily_returns_AGG = df.pct_change()
daily_returns_AGG.head()

,AGG,SPY
,close,close
time,,
2019-11-22 00:00:00-05:00,NaN,NaN
2019-11-25 00:00:00-05:00,0.001152,0.007556
2019-11-26 00:00:00-05:00,0.001239,0.002138
2019-11-27 00:00:00-05:00,-0.001105,0.004586
2019-11-29 00:00:00-05:00,-0.000619,-0.003772


In [7]:
# Calculate volatility
# YOUR CODE HERE
std_dev_daily_return = daily_returns_SPY.std()['close']
std_dev_daily_return
std_dev_daily_return = daily_returns_AGG.std()['close']
std_dev_daily_return
print(joint_df)
variance = df["SPY"].rolling(window=60).var()
variance
covariance = df["SPY"].rolling(window=60).cov(df["AGG"])
beta = covariance/variance
beta.dropna()
print(beta)
beta.plot(figsize=(20, 10), title="SPY, AGG Beta")



KeyError: 'close'

In [9]:
# Save the last day's closing price
# YOUR CODE HERE
SPY_last_price = df['close'][-1]
AGG_last_price = df['close'][-1]

In [10]:
# Setup the Monte Carlo Parameters
number_simulations = 500
number_records = 252 * 30
monte_carlo = pd.DataFrame()

In [11]:
# Run the Monte Carlo Simulation
for x in range(number_simulations):
    
    # YOUR CODE HERE
    simulated_price = simulated_SPY_prices[-1] * (1 + np.random.normal(avg_daily_return, std_dev_daily_return))
    
    simulated_price = simulated_AGG_prices[-1] * (1 + np.random.normal(avg_daily_return, std_dev_daily_return))

In [ ]:
# Visualize the Simulation
# YOUR CODE HERE
monte_carlo.plot(legend=None, title = "Simulated Retirement Portfolio")

In [14]:
# Select the last row for the cumulative returns (cumulative returns at 30 years)
# YOUR CODE HERE
#ending_returns = monte_carlo.iloc[-1, :]
simulated_cumulative_returns = (1 + simulated_daily_returns).cumprod(30)
simulated_cumulative_returns.head([-1])

In [ ]:
# Select the last row for the cumulative returns (cumulative returns at 20 years)
# YOUR CODE HERE
#ending_returns = monte_carlo.tail(1).T
#ending_returns.head()
simulated_cumulative_returns = (1 + simulated_daily_returns).cumprod(20)
simulated_cumulative_returns.head([-1])

In [ ]:
#monte_carlo_t = monte_carlo.transpose()
#monte_carlo_t.head()

In [ ]:
# Display the 90% confidence interval for the ending returns
# YOUR CODE HERE
confidence_interval = freq_dist_df['ending_returns'].quantile(q=[0.900])
confidence_interval

In [ ]:
plt.figure()
freq_dist_df['hit_targets'].plot(kind='hist', density=True, bins=[0,1,2,3,4,5], title="95% Confidence 

In [ ]:
# Visualize the distribution of the ending returns
# YOUR CODE HERE
simulated_ending_returns.plot(kind='hist', bins=20)

---

# Retirement Analysis

In this section, you will use the monte carlo model to answer the following retirement planning questions:

1. What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?
2. Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?
3. Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?
4. How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

### What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?

In [ ]:
# YOUR CODE HERE
initial_investment = 20000
cumulative_returns = initial_investment * simulated_cumulative_returns
cumulative_returns.head()

### Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?

In [ ]:
# YOUR CODE HERE

### Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?

Note: This is effectively saying that 90% of the expected returns will be greater than the return at the 10th percentile, so this can help measure the uncertainty about having enough funds at retirement

In [ ]:
# YOUR CODE HERE

### How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

In [ ]:
# YOUR CODE HERE
initial_investment = 30000
cumulative_returns = initial_investment * simulated_cumulative_returns
cumulative_returns.head()

### Optional Challenge

In this section, you will calculate and plot the cumulative returns for the median and 90% confidence intervals. This plot shows the expected cumulative returns for any given day between the first day and the last day of investment. 

In [ ]:
# YOUR CODE HERE